# Explore signal/price movement relationship

In [ ]:
import datetime
import importlib.util
import sys
from tradingstrategy.client import Client
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.execution_context import notebook_execution_context


from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module

strategies_folder = get_default_strategies_path()
strategy_path = strategies_folder / "spot-and-short-momentum-tuned.py"

strategy_mod = read_strategy_module(strategy_path)

print(f"Backtesting for strategy {strategy_path}, engine version {strategy_mod.trading_strategy_engine_version}")

client = Client.create_jupyter_client()

# contains start_at, end_at backtesting range
universe_options = strategy_mod.get_universe_options()

strategy_universe = strategy_mod.create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    universe_options,
)

data_universe = strategy_universe.data_universe

## Explore signal vs price change



In [44]:
import pandas as pd
import numpy as np
from pandas.tseries.frequencies import to_offset

from tradingstrategy.utils.forward_fill import forward_fill
from tradingstrategy.chain import ChainId


def calculate_signal_vs_profit(df, signal_window: pd.Timedelta, profit_window: pd.Timedelta) -> pd.DataFrame:
    # Create entries for past price (used for signal)
    # and future price (used for the price correlation)

    signal_offset = to_offset(signal_window)
    profit_offset = to_offset(profit_window)

    df["prev"] = df["close"].shift(freq=signal_offset)
    df["next"] = df["open"].shift(freq=-profit_offset)

    df["signal"] = (df["prev"] - df["open"]) / df["open"]
    df["profit"] = (df["next"] - df["open"]) / df["open"]  # Get the profit on the final day of profit window

    df["max_future_price"] = df["close"].rolling(profit_window.days).max().shift(-profit_window.days) # Get the max profit on the profit window, assuming take profit %
    df["profit_max"] = (df["max_future_price"] - df["open"]) / df["open"]  # Get the profit on the final day of profit window
    return df
     

pair = data_universe.pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WETH", "USDC"))
df = data_universe.candles.get_candles_by_pair(pair)
assert df is not None

df = df.copy()

# Make sure there are no gaps in the data
df = forward_fill(
    df, 
    freq=data_universe.time_bucket.to_frequency(), 
    columns=("open", "high", "low", "close")
)

df = calculate_signal_vs_profit(
    df,
    #signal_window=pd.DateOffset(days=7),
    #profit_window=pd.DateOffset(days=7),
    signal_window=pd.Timedelta(days=7),
    profit_window=pd.Timedelta(days=7),
)

pd.set_option('display.min_rows', 10)
display(df)

,open,high,low,close,prev,next,signal,profit,max_future_price,profit_max
timestamp,,,,,,,,,,
2022-07-07,1184.941609,1252.561586,1178.206991,1237.621415,NaN,1114.712532,NaN,-0.059268,1216.882388,0.026956
2022-07-08,1237.497665,1271.872162,1194.458684,1215.787741,NaN,1192.310699,NaN,-0.036515,1230.956604,-0.005286
2022-07-09,1215.544620,1230.956604,1204.654343,1216.882388,NaN,1230.956604,NaN,0.012679,1356.200105,0.115714
2022-07-10,1216.882388,1218.099818,1153.839213,1166.834199,NaN,1356.200105,NaN,0.114487,1356.200105,0.114487
2022-07-11,1166.950882,1169.287002,1091.985483,1096.033110,NaN,1337.747773,NaN,0.146362,1581.982883,0.355655
...,...,...,...,...,...,...,...,...,...,...
2023-10-28,1780.640507,1801.595299,1773.355190,1776.727599,1629.181236,NaN,-0.085059,NaN,NaN,NaN
2023-10-29,1776.727599,1811.168623,1764.510985,1795.121540,1662.922430,NaN,-0.064053,NaN,NaN,NaN
2023-10-30,1795.301052,1830.834264,1779.394557,1809.358450,1765.393417,NaN,-0.016659,NaN,NaN,NaN


## Plot signal vs. price samples

In [45]:
import plotly.express as px
fig = px.scatter(df, x="signal", y="profit_max")
fig.show()

fig = px.scatter(df, x="signal", y="profit")
fig.show()

In [ ]:
# weekday -> period -> pair id

# periods = [6, 7, 8]
# pair_dfs = defaultdict(lambda: defaultdict(int))

# data_df = {}
# for pair in data_universe.pairs.get_all_pair_ids():
#     data_df[pair] = data_universe.candles.get_candles_by_pair(pair)

# for pair in data_universe.pairs.get_all_pair_ids():
#     data = data_df[pair]
#     for day in range(0, 7):
#         for period in periods:
#             pair_dfs[day][period][pair] = calculate_momentum_by_weekday(period, day)
